In [2]:
import os
os.chdir('c:\\Users\\sedar\\Documents\\UCL\\1_BIOL0041_MSci_project\\')
from meshure.utils import list_files_dir_str
import numpy as np 
import matplotlib.pyplot as plt
from meshure.actinimg import get_ActinImg

from ipywidgets import interactive, FloatSlider, IntSlider, HBox, interactive_output

In [ ]:
# list and load files
data_path = os.path.join(os.getcwd(), "actin_meshwork_analysis/process_data/sample_data")
os.path.exists(data_path)

all_filenames, all_filepaths = list_files_dir_str(data_path)
celltype = 'Untransduced'
filename, filepath = all_filenames[celltype][0], all_filepaths[celltype][0] 

# sample image
actimg = get_ActinImg(filename, os.path.join(data_path,celltype))

In [ ]:
def plot_threshold(threshold):
    theta_x6 = np.arange(0,360,60)

    actimg = get_ActinImg(filename, os.path.join(data_path,celltype))
    actimg.normalise()
    actimg.steerable_gauss_2order_thetas(thetas=theta_x6,sigma=2,substack=[1,3],visualise=False)
    actimg.z_project_min()
    actimg.threshold(threshold)

    actimg.visualise('manipulated')

threshold = 0.0001

threshold_widget = FloatSlider(min=0.0001, max=0.009, step=0.0005, value=threshold, readout_format='.5f')
threshold_widget.continuous_update=False

interactive(plot_threshold, threshold=threshold_widget)

In [ ]:
from skimage.measure import profile_line
from matplotlib_scalebar.scalebar import ScaleBar
data_path = os.path.join(os.getcwd(), "actin_meshwork_analysis/process_data/sample_data/CARs")
filename='3min_FOV3_decon.tif'

In [ ]:
def plot_threshold(threshold, theta):

    actimg = get_ActinImg(filename, data_path) # base = [1,4], cyto = [4,7] 
    actimg.normalise()
    actimg.steerable_gauss_2order(theta=theta,sigma=2,substack=[1,4],visualise=False)
    linprof = profile_line(actimg.manipulated_stack, (0,0), actimg.shape)
    linprof = linprof[np.argwhere(linprof>0)]
    thresh = np.percentile(linprof, threshold)
    actimg.z_project_min()
    actimg.threshold(threshold=thresh)
    
    plt.figure()
    plt.subplot(1,1,1)
    plt.imshow(actimg.manipulated_stack,cmap='gray')
    scalebar = ScaleBar(actimg.resolution, 'nm', box_color='None', color='#F2F2F2', location='lower left') 
    plt.gca().add_artist(scalebar)
    plt.axis('off')
    plt.plot((0,actimg.shape[0]),(0,actimg.shape[1]), color='white')
    # plt.subplot(1,2,2)
    # plt.plot(interm) # linprof
    # plt.ylim(0,1.1*np.max(linprof))
    # plt.xlabel('pixel location on line')
    # plt.ylabel('intensity')
    # plt.title('line profile after steerable Gaussian filter')
    # plt.tight_layout()
    

threshold = 0.0001
theta = 0

threshold_widget = IntSlider(min=90, max=99, step=1, value=threshold, continuous_update=False)
theta_widget = FloatSlider(min=0, max=180, step=15, value=theta, readout_format='.0f', continuous_update=False)

interactive(plot_threshold, threshold=threshold_widget, theta=theta_widget)

## Threshold with $\mu$/$\sigma$

In [3]:
data_path = os.path.join(os.getcwd(), "actin_meshwork_analysis/process_data/sample_data/sample_images_threshold")
os.path.exists(data_path)

True

In [8]:
def plot_threshold(factor):
    actimg = get_ActinImg('8min_Thurs_FOV2_decon.tif', data_path) 
    actimg.normalise()
    actimg.steerable_gauss_2order_thetas(thetas=np.linspace(0,180,20),sigma=2,substack=[3,4],visualise=False)
    actimg.z_project_min()

    mu, sigma = actimg.threshold_dynamic(sigma_factor=factor, return_mu_sigma=True)
    actimg.visualise('manipulated', colmap='gray')


factor_widget = FloatSlider(min=0, max=2, step=0.1, value=0, readout_format='.2f')
factor_widget.continuous_update=False

In [9]:
interactive(plot_threshold, factor=factor_widget)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='factor', max=2.0), Output()…

In [8]:
actimg = get_ActinImg('8min_Thurs_FOV2_decon.tif', data_path) 
actimg.normalise()
actimg.steerable_gauss_2order_thetas(thetas=np.linspace(0,180,20),sigma=2,substack=[3,4],visualise=False)
actimg.z_project_min()
actimg.threshold_dynamic(sigma_factor=0, return_mu_sigma=False)

In [18]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
from skimage import data, filters, measure, morphology

img = actimg.manipulated_stack
# Binary image, post-process the binary mask and compute labels
labels = measure.label((img==0).astype('int'))

fig = px.imshow(img, binary_string=True)
fig.update_traces(hoverinfo='skip') # hover is only for label info

props = measure.regionprops(labels, img)
properties = ['area', 'equivalent_diameter', 'eccentricity']

# For each label, add a filled scatter trace for its contour,
# and display the properties of the label in the hover of this trace.
for index in range(1, labels.max()):
    label_i = props[index].label
    contour = measure.find_contours(labels == label_i, 0.25)[0]
    y, x = contour.T
    hoverinfo = ''
    for prop_name in properties:
        hoverinfo += f'<b>{prop_name}: {getattr(props[index], prop_name):.2f}</b><br>'
    fig.add_trace(go.Scatter(
        x=x, y=y, name=label_i,
        mode='lines', fill='toself', showlegend=False,
        hovertemplate=hoverinfo, hoveron='points+fills'))

fig.update_layout(margin=dict(l=2, r=2, t=2, b=2))
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
plotly.io.show(fig)